# Define ModelRegistry

In [1]:
import inspect

class ModelRegistry:
    def __init__(self):
        self.models = {}

    def register_model(self, model_function, input_structure):
        input_mapping = self._determine_input_mapping(model_function, input_structure)
        self.models[model_function.__name__] = {
            'function': model_function,
            'input_mapping': input_mapping
        }

    def _determine_input_mapping(self, model_function, input_structure):
        def find_paths(input_structure, keys):
            """Find the paths to the keys in a nested dictionary."""
            paths = {}

            def recursive_search(current_structure, current_path):
                if isinstance(current_structure, dict):
                    for key, value in current_structure.items():
                        new_path = current_path + [key]
                        if key in keys:
                            paths[key] = '.'.join(new_path)
                            keys.remove(key)
                        if isinstance(value, dict) and keys:
                            recursive_search(value, new_path)

            recursive_search(input_structure, [])
            return paths

        # Extract argument names from the model function
        signature = inspect.signature(model_function)
        keys = list(signature.parameters.keys())
        # Find paths to these keys in the input_structure
        input_mapping = find_paths(input_structure, keys)
        return input_mapping
    
    def get_model(self, model_name):
        if model_name not in self.models:
            raise ValueError(f"Model {model_name} not registered.")
        return self.models[model_name]['function'], self.models[model_name]['input_mapping']


# Define Cow Class

In [2]:
import random
import time
import numpy as np
from datetime import datetime

class Cow:
    def __init__(self, cow_id, input_data):
        self.cow_id = str(cow_id)
        self.input = input_data
        self.results = {}
        self.metadata = {}

    def __getstate__(self):
        state = {
            'cow_id': self.cow_id,
            'input': self.input,
            'results': self.results,
            'metadata': self.metadata
        }
        return state
    
    def __setstate__(self, state):
        self.cow_id = state['cow_id']
        self.input = state['input']
        self.results = state['results']
        self.metadata = state['metadata']

    def run_model(self, model_function, input_mapping):
        start_time = datetime.now()
        result_id = f'{model_function.__name__}_{start_time.date()}'
        inputs = {key: self._get_nested_value(value) for key, value in input_mapping.items()}
        result = model_function(**inputs)
        end_time = datetime.now()
        self.results[result_id] = result

        metadata_entry = {
            'model_name': model_function.__name__,
            'start_time': start_time,
            'end_time': end_time
        }
        self.metadata[result_id] = metadata_entry

    def _get_nested_value(self, path):
        keys = path.split('.')
        value = self.input
        for key in keys:
            value = value[key]
        return value
    
    def _list_results(self):
        print('Result ID'.ljust(25), '| Results')
        for result_id, data in self.results.items():
            print(f'{result_id}'.ljust(25), f'| {data}') 
    
    def _get_result(self, result_id):
        return self.results[result_id]

    def _print_input_structure(self):
        def print_structure(data, indent=0):    
            if isinstance(data, dict):
                for key, value in data.items():
                    print(' ' * indent + str(key))
                    if isinstance(value, dict):
                        print_structure(value, indent + 4)
                    else:
                        print(' ' * (indent + 4) + str(type(value).__name__))
            else:
                print(' ' * indent + str(type(data).__name__))
                
        print_structure(self.input)




    # Testing Methods #
    def moo(self):
        moo_sound = 'M' + 'o' * random.randint(1, 10)
        print(f"{self.cow_id}: {moo_sound}")

    def matrix_multiplication(self, n):
        """Perform an intensive computation with matrix multiplications."""
        start_time = time.time()
        # Create large random matrices
        A = np.random.rand(n, n)
        B = np.random.rand(n, n)
        for _ in range(100):
            C = np.dot(A, B)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"Cow {self.cow_id} performed intensive computation in {elapsed_time:.2f} seconds.")
        self.results['time'] = elapsed_time
        

# Define Herd Class

In [3]:
import json
import pickle
from datetime import datetime

class Herd:
    def __init__(self, name):
        self.name = name
        self.cows_in_herd = []
        self.model_registry = ModelRegistry()
        self.metadata = {}

    def __getstate__(self):
        state = {
            'name': self.name,
            'cows_in_herd': self.cows_in_herd,
            'model_registry': self.model_registry,
            'metadata': self.metadata
        }
        return state

    def __setstate__(self, state):
        self.name = state['name']
        self.cows_in_herd = state['cows_in_herd']
        self.model_registry = state['model_registry']
        self.metadata = state['metadata']

    def add_cow(self, cow):
        self.cows_in_herd.append(cow)

    def load_cows_from_json(self, filename):
        with open(filename, 'r') as file:
            data = json.load(file)
            for cow_data in data:
                cow = Cow(cow_id=cow_data['cow_id'], 
                          input_data=cow_data['input_data'])
                self.add_cow(cow)

    def execute_method(self, method_name, execution_mode='linear', *args, **kwargs):
        # NOTE Used for executing methods of the Cow class
        if execution_mode == 'linear':
            self._execute_linear(method_name, *args, **kwargs)
        elif execution_mode == 'cpu':
            self._execute_cpu(method_name, *args, **kwargs)
        elif execution_mode == 'gpu':
            self._execute_gpu(method_name, *args, **kwargs)
        else:
            raise ValueError("Invalid execution mode. Choose from 'linear', 'cpu', or 'gpu'.")

    def _execute_linear(self, method_name, *args, **kwargs):
        for cow in self.cows_in_herd:
            getattr(cow, method_name)(*args, **kwargs)

    def _execute_cpu(self, method_name, *args, **kwargs):
        raise NotImplementedError("Parallel CPU execution is not implemented yet.")

    def _execute_gpu(self, method_name, *args, **kwargs):
        raise NotImplementedError("GPU execution is not implemented yet.")
    
    def save(self, filename):
        with open(filename, 'wb') as file:
            pickle.dump(self, file)

    @staticmethod
    def load(filename):
        with open(filename, 'rb') as file:
            return pickle.load(file)
        
    def register_model(self, model_function):
        if not self.cows_in_herd:
            raise ValueError("No cows in the herd to determine the input structure.")
        input_structure = self.cows_in_herd[0].input
        self.model_registry.register_model(model_function, input_structure)

    def execute_model(self, model_name, execution_mode='linear'):
        model_function, input_mapping = self.model_registry.get_model(model_name)
        
        if execution_mode == 'linear':
            self._execute_model_linear(model_function, input_mapping)
        elif execution_mode == 'cpu':
            self._execute_model_cpu(model_function, input_mapping)
        elif execution_mode == 'gpu':
            self._execute_model_gpu(model_function, input_mapping)
        else:
            raise ValueError("Invalid execution mode. Choose from 'linear', 'cpu', or 'gpu'.")

    def _execute_model_linear(self, model_function, input_mapping):
        start_time = datetime.now()
        exceptions = {}

        for cow in self.cows_in_herd:
            try:
                cow.run_model(model_function, input_mapping)
            except Exception as e:
                exceptions[cow.cow_id] = e
        
        end_time = datetime.now()
        execution_time = (end_time - start_time).total_seconds()
                
        metadata_entry = {
            'model_name': model_function.__name__,
            'execution_mode': 'linear',
            'start_time': start_time,
            'end_time': end_time,
            'execution_time_seconds': execution_time,
            'errors': exceptions
        }
        self.metadata[f'{model_function.__name__}_{start_time.date()}'] = metadata_entry
        
        if exceptions:
            print('\nThe following Cows failed to run the model: ')
            for cow_id, execption in exceptions.items():
                print(f'{cow_id}: {execption}')

    def _execute_model_cpu(self, model_function, input_mapping):
        raise NotImplementedError("Parallel CPU execution is not implemented yet.")

    def _execute_model_gpu(self, model_function, input_mapping):
        raise NotImplementedError("GPU execution is not implemented yet.")

    def list_cows(self):
        print('Index'.ljust(5), '| Cow ID')
        for index, cow in enumerate(self.cows_in_herd):
            print(f'{index}'.ljust(5), f'| {cow.cow_id}')

    def list_results(self, cow_index):
        cow = self.cows_in_herd[cow_index]
        cow._list_results()

    def get_result(self, cow_index, result_id):
        cow = self.cows_in_herd[cow_index]
        result = cow._get_result(result_id)
        return result

    def check_input(self):
        cow = self.cows_in_herd[0]
        cow._print_input_structure()



# Test Input Mapping

In [4]:
def test_model(a, b, c, d):
    return (a + b / c) ** d

# Create a herd and add cows with nested input data
herd = Herd("Dairy Herd")
herd.add_cow(Cow(cow_id='cow_1', input_data={'level1': {'a': 1, 'b': 2, 'c': 3, 'd': 2}}))
herd.add_cow(Cow(cow_id='cow_2', input_data={'level1': {'a': 2, 'b': 3, 'c': 4, 'd': 3}}))
herd.add_cow(Cow(cow_id='cow_3', input_data={'level1': {'a': 3, 'b': 4, 'c': 5, 'd': 'ads'}}))

# Register the test_model with automatic input mapping
herd.register_model(test_model)

# Execute the model in linear mode
print("Executing test_model in linear mode:")
herd.execute_model('test_model', execution_mode='linear')

# Check results
for cow in herd.cows_in_herd:
    print(f"Cow ID: {cow.cow_id}, Output: {cow.results}")
                

Executing test_model in linear mode:

The following Cows failed to run the model: 
cow_3: unsupported operand type(s) for ** or pow(): 'float' and 'str'
Cow ID: cow_1, Output: {'test_model_2024-05-27': 2.7777777777777772}
Cow ID: cow_2, Output: {'test_model_2024-05-27': 20.796875}
Cow ID: cow_3, Output: {}


In [5]:
herd.list_cows()
print('\n')
herd.list_results(0)


Index | Cow ID
0     | cow_1
1     | cow_2
2     | cow_3


Result ID                 | Results
test_model_2024-05-27     | 2.7777777777777772


In [ ]:
value = herd.get_result(0, 'test_model_2024-05-26')
print(value)


In [ ]:
herd.check_input()

level1
    a
        int
    b
        int
    c
        int
    d
        int


# Test Cow Herd Interaction

In [8]:
test_herd = Herd('test_herd')
test_herd.load_cows_from_json('./demo_cows.json')
# test_herd.execute_function('moo')
# test_herd.execute_function('run_model', 'linear', 'test_model')
        

In [ ]:
for cow in test_herd.cows_in_herd:
    print(f'CowID: {cow.cow_id}, Input Data: {cow.input}')
    

CowID: 1, Input Data: {'age': 2, 'weight': 600, 'milk_yield': 30, 'feed_intake': 25}
CowID: 2, Input Data: {'age': 3, 'weight': 650, 'milk_yield': 28, 'feed_intake': 26}


### Save to File

In [ ]:
test_herd.save('test_herd.pkl')


### Load from File

In [ ]:
loaded_herd = Herd.load('./test_herd.pkl')
loaded_herd.execute_function('moo')
for cow in loaded_herd.cows_in_herd:
    print(f'CowID: {cow.cow_id}, Input Data: {cow.input}')
    


1: Mooooooo
2: Moooooooooo
CowID: 1, Input Data: {'age': 2, 'weight': 600, 'milk_yield': 30, 'feed_intake': 25}
CowID: 2, Input Data: {'age': 3, 'weight': 650, 'milk_yield': 28, 'feed_intake': 26}
